In [1]:
import json
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates


In [10]:
pd.read_csv('../data/mcdw.20230530.csv')

C:\Users\Aaron\AppData\Local\Temp\ipykernel_29936\500318975.py:1: DtypeWarning: Columns (9,13,19) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('../data/mcdw.20230530.csv')


,WMO ID,YEAR-MONTH,LONGITUDE (deg),LATITUDE (deg),ELEVATION (m),STATION PRESSURE (0.1 mb),STATION PRESSURE SOURCE FLAG,MEAN SEA LEVEL PRESSURE (0.1 mb),MEAN SEA LEVEL PRESSURE SOURCE FLAG,MEAN SEA LEVEL PRESSURE HEIGHT FLAG,...,TOTAL PRECIPITATION DEPARTURE (mm),TOTAL PRECIPITATION DEPARTURE SOURCE FLAG,PRECIPITATION QUINTILE,PRECIPITATION QUINTILE SOURCE FLAG,SUNSHINE DURATION (hours),SUNSHINE DURATION SOURCE FLAG,SUNSHINE DURATION AS LONG TERM NORMAL (%),SUNSHINE DURATION AS LONG TERM NORMAL SOURCE FLAG,VERSION,STATION NAME
0,203,2011-06,-9999.0000,-9999.0000,-9999.00,10000,L,10000.0,L,NaN,...,-9999,NaN,-9999,NaN,0,L,0,L,mcdw.20230530,NaN
1,1001,1987-01,-8.6678,70.9397,9.40,10123,L,10135.0,L,NaN,...,5,L,3,L,-9999,NaN,-9999,NaN,mcdw.20230530,JAN MAYEN (1001-1)
2,1001,1987-02,-8.6678,70.9397,9.40,10111,L,10123.0,L,NaN,...,-7,L,3,L,-9999,NaN,-9999,NaN,mcdw.20230530,JAN MAYEN (1001-1)
3,1001,1987-03,-8.6678,70.9397,9.40,10105,L,10117.0,L,NaN,...,19,L,5,L,-9999,NaN,-9999,NaN,mcdw.20230530,JAN MAYEN (1001-1)
4,1001,1987-04,-8.6678,70.9397,9.40,10040,L,10052.0,L,NaN,...,50,L,6,L,-9999,NaN,-9999,NaN,mcdw.20230530,JAN MAYEN (1001-1)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907850,99884,2009-08,-9999.0000,-9999.0000,-9999.00,-9999,NaN,NaN,-9999,NaN,...,-9999,NaN,-9999,NaN,-9999,NaN,-9999,NaN,mcdw.20230530,NaN
907851,99884,2009-09,-9999.0000,-9999.0000,-9999.00,-9999,NaN,NaN,-9999,NaN,...,-9999,NaN,-9999,NaN,-9999,NaN,-9999,NaN,mcdw.20230530,NaN
907852,99999,2020-09,8.9783,46.3361,278.86,-9999,NaN,10000.0,B,NaN,...,-9999,NaN,0,B,0,B,0,B,mcdw.20230530,BIASCA
907853,99999,2021-04,8.9783,46.3361,278.86,-9999,NaN,10000.0,B,NaN,...,-9999,NaN,0,B,0,B,0,B,mcdw.20230530,BIASCA
